<a href="https://colab.research.google.com/github/Ibrah-N/PNID_VLM_Qwen2_Training/blob/main/VLM_SLM_Testing_tanzeem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## pdfs to image

In [9]:
!pip install -q PyMuPDF Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 70.5 MB/s eta 0:00:00


In [10]:
import fitz               # PyMuPDF
from PIL import Image, ImageFilter
import os
import shutil

def pdf_to_jpg(pdf_path, output_folder="output_images", dpi=300,
               threshold=220, output_folder_bw="bw_imgs", pdf_id=0,
               dir_id=0, blur_radius=0.5, image=False
               ):
    """
    Convert each page of a PDF to:
      • color_image_<page>.jpg
      • bw_image_<page>.jpg

    Args:`
      pdf_path (str): path to .pdf file
      output_folder (str): where to save JPEGs
      dpi (int): rendering resolution
      threshold (0–255): gray cutoff for B/W
      blur_radius (float): Gaussian blur radius on B/W
    """
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(output_folder_bw, exist_ok=True)
    doc = fitz.open(pdf_path)
    page_count = doc.page_count

    for i, page in enumerate(doc, start=1):
        pix = page.get_pixmap(dpi=dpi)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # — Color JPEG
        color_file = os.path.join(output_folder, f"color_image_{i}.jpg")
        img.save(color_file, "JPEG", quality=100, dpi=(dpi, dpi))

        # — Grayscale → threshold → blur → B/W JPEG
        gray = img.convert("L")
        bw = gray.point(lambda x: 255 if x > threshold else 0)
        bw = bw.filter(ImageFilter.GaussianBlur(blur_radius))
        # bw_file = os.path.join(output_folder_bw, f"bw_image_{dir_id}_{pdf_id}_{i}.jpg")
        # bw.save(bw_file, "JPEG", quality=100, dpi=(dpi, dpi))

        return bw

    doc.close()
    print(f"Done! For pdf {pdf_id} Saved {page_count} page(s) to '{output_folder}'")

if __name__ == "__main__":
  pass
  # -- single pdf parse --
  # pdf_path = "/content/PNID_VLM/BOM_PDFs/HEA0748-Combined spares manual 100.pdf"
  # pdf_to_jpg(pdf_path,
              # output_folder="/content/output_images")

"""
  # -- iterate over pdf/img files --
  input_dir = "/content/Equipment patches"
  output_bw = "/content/bw_images/"
  os.makedirs(output_bw, exist_ok=True)


  # os.listdir(input_dir)
  # -- iterate over dir --
  for dirIdx, dir in enumerate(os.listdir(input_dir)):
    if dir.endswith("DS_Store"): continue
    dir_path = os.path.join(input_dir, dir)
    print("Processing direcotry: {}".format(dir_path))
    os.makedirs(os.path.join(output_bw, dir), exist_ok=True)
    output_path = os.path.join(output_bw, dir)

    # -- iterate over sub dir --
    for idx, file_x in enumerate(os.listdir(dir_path)):

      # -- check if pdf --
      if (file_x.endswith(".pdf") or file_x.endswith(".PDF")):
        pdf_path = os.path.join(dir_path, file_x)
        pdf_to_jpg(pdf_path,
                  output_folder=output_path,
                  pdf_id = idx,
                  dir_id = dirIdx,
                  output_folder_bw=output_path)

      # -- check if image --
      elif (file_x.endswith(".jpg") or file_x.endswith(".JPG")):
        src_path = os.path.join(dir_path, file_x)
        dst_path = os.path.join(output_path, file_x)
        print(src_path, dst_path)
        shutil.copy(src_path, dst_path)
        print(f"Done! For :{file_x} Copied to {dst_path}")

      # -- else skip --
      else:
        print("Not a pdf or jpeg file -- file {}".format(file_x))
"""

'\n  # -- iterate over pdf/img files --\n  input_dir = "/content/Equipment patches"\n  output_bw = "/content/bw_images/"\n  os.makedirs(output_bw, exist_ok=True)\n\n\n  # os.listdir(input_dir)\n  # -- iterate over dir --\n  for dirIdx, dir in enumerate(os.listdir(input_dir)):\n    if dir.endswith("DS_Store"): continue\n    dir_path = os.path.join(input_dir, dir)\n    print("Processing direcotry: {}".format(dir_path))\n    os.makedirs(os.path.join(output_bw, dir), exist_ok=True)\n    output_path = os.path.join(output_bw, dir)\n\n    # -- iterate over sub dir --\n    for idx, file_x in enumerate(os.listdir(dir_path)):\n\n      # -- check if pdf --\n      if (file_x.endswith(".pdf") or file_x.endswith(".PDF")):\n        pdf_path = os.path.join(dir_path, file_x)\n        pdf_to_jpg(pdf_path,\n                  output_folder=output_path,\n                  pdf_id = idx,\n                  dir_id = dirIdx,\n                  output_folder_bw=output_path)\n\n      # -- check if image --\n    

# Parse Data [DATA VERIFICATION]
  - Load Data
  - Parse Sonnet Data
    - Images | JSONs
  
  - Parse Genereated CSV
    - Images | CSVs

## Load Data

In [1]:
from google.colab import drive
drive.mount("/content/ibrah_drive")

Mounted at /content/ibrah_drive


In [2]:
# Copy the BOM Data Into local drive


import os

global_path = "/content/ibrah_drive/MyDrive/PNID_VLM"
local_path = "PNID_VLM"

os.makedirs(local_path, exist_ok=True)

# -- copy to local path --
!cp -r {global_path}/* {local_path}

In [3]:
# -- unzip the zipped folder --
!unzip -q {local_path}/BOM_PDFs.zip -d {local_path}
!unzip -q {local_path}/BOM_Output_json.zip -d {local_path}
!unzip -q {local_path}/BOM_CSV_Individual.zip -d {local_path}

In [4]:
# configs

import os


pdfs_path = "/content/PNID_VLM/BOM_PDFs"
jsons_path = "/content/PNID_VLM/BOM_Outputs"
csvs_path = "/content/PNID_VLM/BOM_CSV_Individual"
new_csvs_path = "/content/PNID_VLM/BOM_CSV_Corrected"

os.makedirs(new_csvs_path, exist_ok=True)

## Parse Sonnet Generetted JSON and Image

In [ ]:
# parse the json date extracted
# directly from the sonnet 4 model

import json
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
pd.set_option('display.max_columns', None)


for idx, pdf in enumerate(os.listdir(pdfs_path)):
  pdf_path = os.path.join(pdfs_path, pdf)
  json_filename = pdf + ".json" # because json name is xyz.pdf.json
  json_path = os.path.join(jsons_path, json_filename)

  print("PDF path: {} - json_path {}".format(pdf_path, json_path))

  # -- if json not exists aginst pdf --
  if not os.path.exists(json_path):
    print(f"Warning: JSON file not found for PDF: {pdf}. Skipping. Expected path: {json_path}")
    continue

  # -- read pdf and json --
  img_data = pdf_to_jpg(pdf_path)
  img_data = np.array(img_data)
  with open(json_path, 'r') as f:
    json_data = json.load(f)
  table_data = pd.DataFrame(json_data['rows'])

  if idx == 0:
    break

PDF path: /content/PNID_VLM/BOM_PDFs/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 941.pdf - json_path /content/PNID_VLM/BOM_Outputs/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 941.pdf.json


In [ ]:
cv2_imshow(img_data)

In [ ]:
table_data.head()

## Parse The Converted CSV

In [6]:
# ---- naming json, csvs and pdfs ----

ashwaryas_csv_naming = os.listdir(csvs_path)
# naming sequence is corrected; now matching with the names of pdf
# previosuly to the names mistakenly added by the editor this charachter "_"
ashwaryas_corrected_csv_naming = [file_name.replace("_", " ") for file_name in ashwaryas_csv_naming]
vinod_names = os.listdir(pdfs_path)

print("Vinod Names      : ", vinod_names)
print("Ashwaryas Names  : ", ashwaryas_csv_naming)
print("Correct Ash Names: ",ashwaryas_corrected_csv_naming)

Vinod Names      :  ['VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 877.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 1041.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 1009.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 835.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 849.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 1105.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 867.pdf', 'HEA0748-Combined spares manual 66.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 891.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 865.pdf', 'HEA0748-Combined spares manual 110.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 1083.pdf', 'HEA0748-Combined spares manual 98.pdf', 'HEA0748-Combined spares manual 48.pdf', 'HEA0748-Combined spares manual 34.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Ca

In [7]:
# -- move the corrected csvs into new directory --

import shutil

for idx in range(len(ashwaryas_corrected_csv_naming)):
  old_csv_name = ashwaryas_csv_naming[idx] # files_names: are old csv with wrong names
  new_csv_name = ashwaryas_corrected_csv_naming[idx] # correct_file_names: correct names for csvs matching with pdfs

  csv_path = os.path.join(csvs_path, old_csv_name)
  new_csv_path = os.path.join(new_csvs_path, new_csv_name)

  shutil.copy(csv_path, new_csv_path)

In [8]:
print(sorted(ashwaryas_csv_naming))
print(sorted(ashwaryas_corrected_csv_naming))

['HEA0748-Combined_spares_manual_100.csv', 'HEA0748-Combined_spares_manual_102.csv', 'HEA0748-Combined_spares_manual_104.csv', 'HEA0748-Combined_spares_manual_106.csv', 'HEA0748-Combined_spares_manual_108.csv', 'HEA0748-Combined_spares_manual_11.csv', 'HEA0748-Combined_spares_manual_110.csv', 'HEA0748-Combined_spares_manual_112.csv', 'HEA0748-Combined_spares_manual_114.csv', 'HEA0748-Combined_spares_manual_116.csv', 'HEA0748-Combined_spares_manual_118.csv', 'HEA0748-Combined_spares_manual_120.csv', 'HEA0748-Combined_spares_manual_122.csv', 'HEA0748-Combined_spares_manual_124.csv', 'HEA0748-Combined_spares_manual_13.csv', 'HEA0748-Combined_spares_manual_15.csv', 'HEA0748-Combined_spares_manual_17.csv', 'HEA0748-Combined_spares_manual_19.csv', 'HEA0748-Combined_spares_manual_21.csv', 'HEA0748-Combined_spares_manual_23.csv', 'HEA0748-Combined_spares_manual_25.csv', 'HEA0748-Combined_spares_manual_30.csv', 'HEA0748-Combined_spares_manual_32.csv', 'HEA0748-Combined_spares_manual_34.csv', 'H

## Implementation

In [16]:
# parse the csv date converted
# into csv for correction

import pandas as pd
import numpy as np
import os
from google.colab.patches import cv2_imshow
pd.set_option('display.max_columns', None)



#======================================#
#   R E A D - C S V - F U N C T I O N  #
#======================================#
def read_csv(csv_path):
  """Read csv of these three formats, utf-8, latin1, cp1252

  Args:
      csv_path (str): path to csv file

  Returns:
      df: pandas dataframe
  """
  try:
      df = pd.read_csv(csv_path, encoding='utf-8')
  except UnicodeDecodeError:
      print("UTF-8 decoding failed, trying 'latin1'...")
      try:
          df = pd.read_csv(csv_path, encoding='latin1')
      except UnicodeDecodeError:
          print("Latin1 decoding also failed, trying 'cp1252'...")
          df = pd.read_csv(csv_path, encoding='cp1252')
  # return
  return df



id = 4
for idx, pdf in enumerate(os.listdir(pdfs_path)):
  pdf_path = os.path.join(pdfs_path, pdf)
  csv_file_name = pdf.replace(".pdf", ".csv") # because json/csv name is xyz.pdf --> xyz.csv
  csv_path = os.path.join(new_csvs_path, csv_file_name)

  if not idx == id:
    continue
  print("PDF path: {} - CSV path {}".format(pdf_path, csv_path))


  # -- if csv not exists aginst pdf --
  if not os.path.exists(csv_path):
    print(f"Warning: CSV file not found for PDF: {pdf}. Skipping. Expected path: {csv_path}")
    continue

  # -- read pdf and csv --
  img_data = pdf_to_jpg(pdf_path)
  img_data = np.array(img_data)
  table_data_df = read_csv(csv_path)


  # Problem: 5th, 6th, 8th-9th, 10th(material specification), 14, 101(not sure, remarks), 102(material specification issue)
  if idx == id:
    break

PDF path: /content/PNID_VLM/BOM_PDFs/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 849.pdf - CSV path /content/PNID_VLM/BOM_CSV_Corrected/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 849.csv
UTF-8 decoding failed, trying 'latin1'...


In [ ]:
table_data_df.head(20)

In [ ]:
cv2_imshow(img_data)

# Transforming Data
  - Utils
  - Path Configs
  - Transformation: CSV 2 JSON

## Utils
  - read_csv function
  - pdf to image function

In [ ]:
!pip install -q PyMuPDF Pillow

In [ ]:
#======================================#
#   R E A D - C S V - F U N C T I O N  #
#======================================#
import pandas as pd

def read_csv(csv_path):
  """Read csv of these three formats, utf-8, latin1, cp1252

  Args:
      csv_path (str): path to csv file

  Returns:
      df: pandas dataframe
  """
  try:
      df = pd.read_csv(csv_path, encoding='utf-8')
  except UnicodeDecodeError:
      print("UTF-8 decoding failed, trying 'latin1'...")
      try:
          df = pd.read_csv(csv_path, encoding='latin1')
      except UnicodeDecodeError:
          print("Latin1 decoding also failed, trying 'cp1252'...")
          df = pd.read_csv(csv_path, encoding='cp1252')
  # return
  return df

In [ ]:
import fitz               # PyMuPDF
from PIL import Image, ImageFilter
import os
import shutil

def pdf_to_jpg(pdf_path, output_folder="output_images", dpi=300,
               threshold=220, output_folder_bw="bw_imgs", pdf_id=0,
               dir_id=0, blur_radius=0.5, image=False
               ):
    """
    Convert each page of a PDF to:
      • color_image_<page>.jpg
      • bw_image_<page>.jpg

    Args:`
      pdf_path (str): path to .pdf file
      output_folder (str): where to save JPEGs
      dpi (int): rendering resolution
      threshold (0–255): gray cutoff for B/W
      blur_radius (float): Gaussian blur radius on B/W
    """
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(output_folder_bw, exist_ok=True)
    doc = fitz.open(pdf_path)
    page_count = doc.page_count

    for i, page in enumerate(doc, start=1):
        pix = page.get_pixmap(dpi=dpi)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # — Color JPEG
        color_file = os.path.join(output_folder, f"color_image_{i}.jpg")
        img.save(color_file, "JPEG", quality=100, dpi=(dpi, dpi))

        # — Grayscale → threshold → blur → B/W JPEG
        gray = img.convert("L")
        bw = gray.point(lambda x: 255 if x > threshold else 0)
        bw = bw.filter(ImageFilter.GaussianBlur(blur_radius))
        # bw_file = os.path.join(output_folder_bw, f"bw_image_{dir_id}_{pdf_id}_{i}.jpg")
        # bw.save(bw_file, "JPEG", quality=100, dpi=(dpi, dpi))

        return bw

    doc.close()
    print(f"Done! For pdf {pdf_id} Saved {page_count} page(s) to '{output_folder}'")

if __name__ == "__main__":
  pass
  # -- single pdf parse --
  # pdf_path = "/content/PNID_VLM/BOM_PDFs/HEA0748-Combined spares manual 100.pdf"
  # pdf_to_jpg(pdf_path,
              # output_folder="/content/output_images")

## Path Configs

In [ ]:
from google.colab import drive
drive.mount("/content/ibrah_drive")

In [ ]:
# -- raw paths configs --

import os

# -- local read configuration --
pdfs_path = "/content/PNID_VLM/BOM_PDFs"
jsons_path = "/content/PNID_VLM/BOM_Outputs"
csvs_path = "/content/PNID_VLM/BOM_CSV_Individual"
new_csvs_path = "/content/PNID_VLM/BOM_CSV_Corrected"

os.makedirs(new_csvs_path, exist_ok=True)


# -- NEW DATASET: local write configuration --
root_write_path = "VLM_Dataset"
root_write_img_path = os.path.join(root_write_path, "images")
root_write_json_path = os.path.join(root_write_path, "json")

os.makedirs(root_write_img_path, exist_ok=True)
os.makedirs(root_write_json_path, exist_ok=True)

In [ ]:
# ---- naming json, csvs and pdfs ----

ashwaryas_csv_naming = os.listdir(csvs_path)
# naming sequence is corrected; now matching with the names of pdf
ashwaryas_corrected_csv_naming = [file_name.replace("_", " ") for file_name in ashwaryas_csv_naming]
vinod_names = os.listdir(pdfs_path)

print("Vinod Names      : ", vinod_names)
print("Ashwaryas Names  : ", ashwaryas_csv_naming)
print("Correct Ash Names: ",ashwaryas_corrected_csv_naming)

In [ ]:
# -- move the csvs into new directory --

import shutil

for idx in range(len(ashwaryas_corrected_csv_naming)):
  old_csv_name = ashwaryas_csv_naming[idx] # files_names: are old csv with wrong names
  new_csv_name = ashwaryas_corrected_csv_naming[idx] # correct_file_names: correct names for csvs matching with pdfs

  csv_path = os.path.join(csvs_path, old_csv_name)
  new_csv_path = os.path.join(new_csvs_path, new_csv_name)

  shutil.copy(csv_path, new_csv_path)

## CSV 2 JSON

In [ ]:
#===================================================#
#   C S V - T O - J S O N; P D F - T O - I M A G E  #
#===================================================#
# reads the csv and pdf to jpg
# drop the column file_name
# convert the csv to json
# save each json into new_directory


# -- imports --
import json
import pandas as pd
import os


# -- read csv and pdf --
for pdf_idx, pdf_name in enumerate(os.listdir(pdfs_path)):

  # - pdf and csv names -
  pdf_file_name = pdf_name
  csv_file_name = pdf_name.replace(".pdf", ".csv") # because json/csv name is xyz.pdf --> xyz.csv
  img_file_name = pdf_name.replace(".pdf", ".jpg")
  json_file_name = pdf_name.replace(".pdf", ".json")

  # - read pdf and csv path configuration -
  pdf_path = os.path.join(pdfs_path, pdf_file_name)
  csv_path = os.path.join(new_csvs_path, csv_file_name)

  # - write img and csv configuration -
  img_write_path = os.path.join(root_write_img_path, img_file_name)
  json_write_path = os.path.join(root_write_json_path, json_file_name)


  # - checks conditions -
  if not os.path.exists(csv_path):
    print(f"Warning: CSV file not found for PDF: {pdf_name}. Skipping. Expected path: {csv_path}")
    continue

  # - read pdf and csv -
  img_data = pdf_to_jpg(pdf_path)
  table_data_df = read_csv(csv_path)

  # - transform csv data -
  # drop file name
  # convert nan to ""
  # convert to json
  table_data_df = table_data_df.drop(columns=["file_name", "sheet_name"])
  table_data_df = table_data_df.fillna("")
  table_data_json = '{{"rows" : {}}}'.format(table_data_df.to_json(orient='records'))


  # - write image and json -
  # image
  img_data.save(img_write_path, "JPEG", quality=100, dpi=(300, 300))
  # json
  with open(json_write_path, 'w') as f:
    json.dump(json.loads(table_data_json), f, indent=4)

  print(f"No {pdf_idx} - Image Saved to: {img_write_path} - JSON Saved to: {json_write_path}")


In [ ]:
# copy the VLM_Dataset to drive

!cp /content/VLM_Dataset/* -r /content/ibrah_drive/MyDrive/PNID_VLM/VLM_Dataset/

In [ ]:
# -- GitHub Commits --
# Repo Name: https://github.com/Ibrah-N/PNID_VLM_Qwen2_Training

# COMMITS:
#  1. Data Verification Pipeline. [DONE]
#  2. Data Transformation Pipeline; Converting data from CSVs to JSONs. [DONE]

# Data Representation

## Global Configs

In [1]:
system_message = '''You are an expert vision-language document extraction system.

Your ONLY task is to extract Bill of Materials (BOM) data from images of tables, drawings, PDFs, or scanned documents and return it as a STRICTLY VALID JSON object.

You must be:
- STRICT
- CONSERVATIVE
- CONSISTENT

NEVER guess or infer missing values.
If a value is not clearly visible or labeled, output an empty string "".

You must extract EXACTLY the following 12 fields for each row:
1. Position Number
2. Tag Number
3. Description of parts
4. Material Specification
5. Quantity
6. Manufacturer's drawing or Ref.No.
7. Original Equipment Manufacturer
8. Original Part Manufacturer
9. Manufacturer's real part No.
10. Unit of Measurement (UOM)
11. Approximate unit price in SAR
12. Recommended Quantity

STRICT RULES:
- NEVER map Part Number, Code, Parça No, or Item Code to UOM.
- NEVER invent Tag Numbers or UOM values.
- If there is no explicit Tag or UOM column, leave those fields empty.
- If you are unsure, leave the field empty "".

Your final output MUST:
- Be valid JSON
- Contain ONLY the JSON object
- Match the exact schema provided
- Contain no explanations, comments, or formatting outside JSON
'''

In [2]:
query_prompt = '''Extract all visible Bill of Materials (BOM) rows from the image.

Return the result as a valid JSON object in EXACTLY this structure:

{
  "rows": [
    {
      "Position Number": "",
      "Tag Number": "",
      "Description of parts": "",
      "Material Specification": "",
      "Quantity": "",
      "Manufacturer's drawing or Ref.No.": "",
      "Original Equipment Manufacturer": "",
      "Original Part Manufacturer": "",
      "Manufacturer's real part No.": "",
      "Unit of Measurement (UOM)": "",
      "Approximate unit price in SAR": "",
      "Recommended Quantity": ""
    }
  ]
}

Rules:
- Include a row ONLY if Position Number is present.
- Do NOT guess or infer values.
- If a value is missing or unclear, use an empty string "".
- Output ONLY the JSON object. No explanations.
'''

## Utils

In [3]:

import os
import json
from PIL import Image

#===========================================#
#    R E P R E S E N T - D A T A S E T      #
#===========================================#
# -> take images and josn to be converted into dict
def represent_dataset(root_images, root_jsons, root_images_path, root_jsons_path, prompt):
  """It takes the required params and aligns it accordingly; meanwhile images_path, jsons_paths, prompt

  Args:
      root_images (str): list of images in the root_images_path
      root_jsons (str): list of jsons in the root_jsons_path path
      root_images_path (str): A path to the root_images
      root_jsons_path (str): A path to the root jsons_path
      prompt (str): prompt to be attaches to the sample

  Return:
    samples_list; [{"prompt": , "image_path": , "json_path": }]
  """
  samples_list = []

  for idx in range(len(root_images)):
    img_sample = sorted(root_images)[idx]
    json_sample = sorted(root_jsons)[idx]

    img_sample_path = os.path.join(root_images_path, img_sample)
    json_sample_path = os.path.join(root_jsons_path, json_sample)

    sample_dict = {
        "prompt": prompt,
        "image_path": img_sample_path,
        "json_path": json_sample_path
    }

    samples_list.append(sample_dict)

  return samples_list

In [4]:
#===========================================#
#       L O A D - I M A G E & J S O N       #
#===========================================#
# -> functions load image and json

def load_json(json_path):
  """load the json from the path

  Args:
      json_path (str): path to the json file

  Returns:
      loaded_json: loaded json file
  """

  with open(json_path, 'r') as f:
    loaded_json = json.load(f)


  # IMPORTANT: keep JSON formatted
  structure_text = json.dumps(
      loaded_json,
      ensure_ascii=False,
      indent=2
  )
  return structure_text


def load_image(image_path):
  """load the image from the path

  Args:
      image_path (str): path to the image file

  Returns:
      loaded_image: loaded image file
  """

  loaded_image = Image.open(image_path).convert("RGB")

  return loaded_image

In [5]:
#===========================================#
#       L O A D - D A T A S E T.            #
#===========================================#
# -> load the images and jsons into dataset

def load_dataset(sample_dict):
  """load the images and jsons into dataset

  Args:
    sample_dict (dict): {"prompt": , "image_path": , "json_path": }

  Returns:
    loaded_dataset: prompt, PIL Image, Loaded Json
  """

  prompt = sample_dict["prompt"]
  image_path = sample_dict["image_path"]
  json_path = sample_dict["json_path"]

  loaded_image = load_image(image_path)
  loaded_json = load_json(json_path)

  return {"prompt": prompt, "image": loaded_image, "label": loaded_json}


In [6]:
#===========================================#
#   F O R M A T E - T H E - S A M P L E     #
#===========================================#
# -> formating each sample for the model
# training inputs
def format_sample(sample):

  return {
      "images": [sample['image']],
      "messages": [
          # system
          {
              "role": "system",
              "content": [
                  {
                      "type": "text",
                      "text": system_message
                  }
              ],
          },

          # user
          {
              "role": "user",
              "content": [
                  {
                      "type": "image",
                      "image": sample['image']
                  },
                  {
                      "type": "text",
                      "text": sample['prompt']
                  }
              ],
          },

          # assistant
          {
              "role": "assistant",
              "content": [
                  {
                      "type": "text",
                      "text": sample['label']
                  }
              ],
          },

      ]
  }

## Paths Configs

In [7]:
from google.colab import drive
drive.mount("/content/ibrah_drive")

Mounted at /content/ibrah_drive


In [8]:
# copy data from drive into local folder

import os

global_path = "/content/ibrah_drive/MyDrive/PNID_VLM/VLM_Dataset"
local_path = "/content/PNID_VLM"

os.makedirs(local_path, exist_ok=True)

!cp -r {global_path} -d {local_path}
print(f"Data copied from {global_path} to {local_path}")

Data copied from /content/ibrah_drive/MyDrive/PNID_VLM/VLM_Dataset to /content/PNID_VLM


In [9]:
# -- dataset paths configuration --

root = "/content/PNID_VLM/VLM_Dataset"
root_images = os.path.join(root, "images")
root_jsons = os.path.join(root, "json")


print("**Dataset Paths**")
print(f"Root Path: {root} \nImages Path: {root_images} \nJSONs Path: {root_jsons}")

**Dataset Paths**
Root Path: /content/PNID_VLM/VLM_Dataset 
Images Path: /content/PNID_VLM/VLM_Dataset/images 
JSONs Path: /content/PNID_VLM/VLM_Dataset/json


## Implementation

In [10]:
# -- create train & valid samples --
# train split: 90%
# valid split: 10%


# -- imports --
import random

# -- collect samples --
list_images = os.listdir(root_images)
list_jsons = os.listdir(root_jsons)

print(f"Total Images: {len(list_images)} - Total Jsons: {len(list_jsons)}")


# -- shuffles the lists --
random.shuffle(list_images)


# -- ratios --
train_split = 0.9
valid_split = 0.1
total_samples = len(list_images)
train_samples = int(total_samples * train_split) # 90%
valid_samples = int(total_samples * valid_split) # 10%


# -- splits --
# images
train_images = list_images[:train_samples]
valid_images = list_images[train_samples:]
# jsons
train_jsons = [image_name.replace(".jpg", ".json") for image_name in train_images]
valid_jsons = [image_name.replace(".jpg", ".json") for image_name in valid_images]


# -- checks --
# train
for idx, image_name in enumerate(train_images):
  json_name = image_name.replace(".jpg", ".json")
  # dual checks for the existance of corrospendent json
  if not json_name in train_jsons or json_name not in list_jsons:
    print(f"Warning: Json {json_name} not found in list_jsons or train_jsons")
# valid
for idx, image_name in enumerate(valid_images):
  json_name = image_name.replace(".jpg", ".json")
  # dual checks for the existance of corrospendent json
  if not json_name in valid_jsons or json_name not in list_jsons:
    print(f"Warning: Json {json_name} not found in list_jsons or valid_jsons")



print("train images", len(train_images), sorted(train_images))
print("train jsons", len(train_jsons), sorted(train_jsons))
print("valid images", len(valid_images), sorted(valid_images))
print("valid jsons", len(valid_images), sorted(valid_jsons))

Total Images: 196 - Total Jsons: 196
train images 176 ['HEA0748-Combined spares manual 100.jpg', 'HEA0748-Combined spares manual 102.jpg', 'HEA0748-Combined spares manual 104.jpg', 'HEA0748-Combined spares manual 106.jpg', 'HEA0748-Combined spares manual 108.jpg', 'HEA0748-Combined spares manual 11.jpg', 'HEA0748-Combined spares manual 110.jpg', 'HEA0748-Combined spares manual 114.jpg', 'HEA0748-Combined spares manual 118.jpg', 'HEA0748-Combined spares manual 120.jpg', 'HEA0748-Combined spares manual 122.jpg', 'HEA0748-Combined spares manual 124.jpg', 'HEA0748-Combined spares manual 13.jpg', 'HEA0748-Combined spares manual 15.jpg', 'HEA0748-Combined spares manual 17.jpg', 'HEA0748-Combined spares manual 19.jpg', 'HEA0748-Combined spares manual 21.jpg', 'HEA0748-Combined spares manual 23.jpg', 'HEA0748-Combined spares manual 25.jpg', 'HEA0748-Combined spares manual 30.jpg', 'HEA0748-Combined spares manual 32.jpg', 'HEA0748-Combined spares manual 34.jpg', 'HEA0748-Combined spares manual 

In [11]:
#===============================================#
#   R E P R E S E N T - R A W - D A T A S E T   #
#===============================================#
# the seperate images, jsons and paths are aligns
# into singuler list of dict along witht the prompt

# -- imports --
# from datasets import Dataset # for Graph model only



# -- represent dataset --
# represent:
#     images_path: [img_1.jpg, img_2.jpg ....],
#     jsons_path: [img_1.json, img_2.json, ....],
# into ---->
#     [{"prompt", "img_path": /image/img_1.jpg, "json_path": /json/img_1.json}
#      {"prompt", "img_path": /image/img_2.jpg, "json_path": /json/img_2.json}
#       ......
#     ]
training_samples = represent_dataset(train_images, train_jsons, root_images, root_jsons, query_prompt)
valid_samples = represent_dataset(valid_images, valid_jsons, root_images, root_jsons, query_prompt)



# -- load the dataset hf dataset --
# from eager mode into graph mode
# training_samples = Dataset.from_list(training_samples)
# valid_samples = Dataset.from_list(valid_samples)

# -- load the dataset images and jsons--
# load images_paths into PIL object
# load jsons_paths into json objects
#     [{"prompt", "img_path": PIL.image, "json_path": json_object}
#      {"prompt", "img_path": PIL.image, "json_path": json_object}
#       ......
#     ]
# = Graph Mode =
# training_samples = training_samples.map(
#     load_dataset,
#     remove_columns=training_samples.column_names
# )
# valid_samples = valid_samples.map(
#     load_dataset,
#     remove_columns=valid_samples.column_names
# )
# = Eager Mode =
load_train_dataset = [load_dataset(sample_dict) for sample_dict in training_samples]
load_valid_dataset = [load_dataset(sample_dict) for sample_dict in valid_samples]


# -- format the samples for QwenVL--
# convert dataset raw samples into
# model structured samples
# = Graph Mode =
# train_dataset = training_samples.map(
#     format_sample,
#     remove_columns=training_samples.column_names
# )
# valid_dataset = valid_samples.map(
#     format_sample,
#     remove_columns=valid_samples.column_names
# )
# = Eager Mode =
train_dataset = [format_sample(sample_dict) for sample_dict in load_train_dataset]
valid_dataset = [format_sample(sample_dict) for sample_dict in load_valid_dataset]



# -- consoles --
print(f"Training Samples: {len(train_dataset)}")
print(f"Validation Samples: {len(valid_dataset)}")
print(f"\n========================================")
print(f"Training Sample: {train_dataset[0]}")
print(f"\n========================================")
print(f"Training Sample: {valid_dataset[0]}")

Training Samples: 176
Validation Samples: 20

Training Sample: {'images': [<PIL.Image.Image image mode=RGB size=2481x3509 at 0x78BE129296D0>], 'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': 'You are an expert vision-language document extraction system.\n\nYour ONLY task is to extract Bill of Materials (BOM) data from images of tables, drawings, PDFs, or scanned documents and return it as a STRICTLY VALID JSON object.\n\nYou must be:\n- STRICT\n- CONSERVATIVE\n- CONSISTENT\n\nNEVER guess or infer missing values.\nIf a value is not clearly visible or labeled, output an empty string "".\n\nYou must extract EXACTLY the following 12 fields for each row:\n1. Position Number\n2. Tag Number\n3. Description of parts\n4. Material Specification\n5. Quantity\n6. Manufacturer\'s drawing or Ref.No.\n7. Original Equipment Manufacturer\n8. Original Part Manufacturer\n9. Manufacturer\'s real part No.\n10. Unit of Measurement (UOM)\n11. Approximate unit price in SAR\n12. Recommende

In [20]:
def hf_upload(sample_dict):
  return {
      "image": sample_dict["image"],
      "query": sample_dict["prompt"],
      "label": [sample_dict["label"]]
  }

In [27]:
hf_uploadf_train_dataset = [hf_upload(sample_dict) for sample_dict in load_train_dataset]
hf_upload_valid_dataset = [hf_upload(sample_dict) for sample_dict in load_valid_dataset]

In [ ]:
!pip install -q huggingface_hub datasets


In [31]:
from huggingface_hub import login

# This will ask you to enter your Hugging Face API token
login()

In [28]:
from datasets import Dataset, Features, Image, Value

features = Features({
    "image": Image(),
    "query": Value("string"),
    "label": Value("string")
})

hf_train_dataset = Dataset.from_list(hf_uploadf_train_dataset, features=features)
hf_valid_dataset = Dataset.from_list(hf_upload_valid_dataset, features=features)

In [29]:
from datasets import DatasetDict

hf_dataset = DatasetDict({
    "train": hf_train_dataset,
    "validation": hf_valid_dataset
})


In [32]:
dataset_name = "Ibrah-N/bom_dataset"  # change this to your HF repo name

hf_dataset.push_to_hub(dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Map:   0%|          | 0/176 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :  88%|########8 | 3.75MB / 4.25MB            

CommitInfo(commit_url='https://huggingface.co/datasets/Ibrah-N/bom_dataset/commit/83b59bebdce57f6ac04270f2bd77142ec2f5bdd6', commit_message='Upload dataset', commit_description='', oid='83b59bebdce57f6ac04270f2bd77142ec2f5bdd6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Ibrah-N/bom_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Ibrah-N/bom_dataset'), pr_revision=None, pr_num=None)

In [33]:
from datasets import load_dataset

dataset = load_dataset(dataset_name)

README.md:   0%|          | 0.00/465 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/39.1M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/4.25M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20 [00:00<?, ? examples/s]

# Experiments

In [ ]:
import pandas as pd


csv_p = "/content/PNID_VLM/BOM_CSV_Corrected/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 837.csv"

df = read_csv(csv_p)
df = df.fillna("")
df = df.drop(columns=["file_name", "sheet_name"])
df.head()

In [ ]:
df.to_json("json_xyz.json", orient='records')

In [ ]:
x = df.to_json(orient='records')
json_x = '{{"row" : {}}}'.format(x)
print(json_x)

{"row" : [{"Position Number":1,"Tag Number":"","Description of parts":"Body","Material Specification":"","Quantity":1,"Manufacturer's drawing or Ref.No.":"T1156-143-01","Original Equipment Manufacturer":"Titan","Original Part Manufacturer":"","Manufacturer's real part No.":"T1156-144-01","Unit of Measurement (UOM)":"","Approximate unit price in SAR":"","Recommended Quantity":""},{"Position Number":2,"Tag Number":"","Description of parts":"Circlip","Material Specification":"13x1","Quantity":2,"Manufacturer's drawing or Ref.No.":"T1156-143-01","Original Equipment Manufacturer":"Titan","Original Part Manufacturer":"","Manufacturer's real part No.":"T103-04-012","Unit of Measurement (UOM)":"","Approximate unit price in SAR":"","Recommended Quantity":""},{"Position Number":3,"Tag Number":"","Description of parts":"Castermid","Material Specification":"","Quantity":2,"Manufacturer's drawing or Ref.No.":"T1156-143-01","Original Equipment Manufacturer":"Titan","Original Part Manufacturer":"","M

In [ ]:
import json

# Define the output path for the JSON file
output_json_path = os.path.join(jsons_path, pdf.replace('.pdf', '_corrected.json'))

# Parse the string json_x back to a Python object, then write it to file
with open(output_json_path, 'w') as f:
    json.dump(json.loads(json_x), f, indent=4)

print(f"JSON data saved to: {output_json_path}")

JSON data saved to: /content/PNID_VLM/BOM_Outputs/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 837_corrected.json
